## Environment setup and installations

In [ ]:
!pip install git+https://github.com/openai/whisper.git

In [ ]:
!pip install transformers datasets tokenizers
import io
import os
import zipfile
import requests
import pandas as pd
import torch
import whisper
from transformers import AutoTokenizer, AutoModelForMaskedLM, pipeline
# Load the BERT model and tokenizer for corrections
from transformers import BertTokenizer, BertForMaskedLM, pipeline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Loading data

In [ ]:
class AsantiTwiDataset(torch.utils.data.Dataset):
    def __init__(self, zip_url, csv_filename, audio_base_path, processor=None, device="cpu"):
        self.device = device
        self.audio_base_path = audio_base_path
        self.processor = processor

        # Download and extract the dataset
        response = requests.get(zip_url, stream=True)
        response.raise_for_status()
        with zipfile.ZipFile(io.BytesIO(response.content), 'r') as zip_ref:
            zip_ref.extractall('.')

        # Clean the CSV file
        cleaned_csv_filename = f"cleaned_{os.path.basename(csv_filename)}"
        self.clean_csv(csv_filename, cleaned_csv_filename)

        # Load the cleaned CSV
        self.df = pd.read_csv(cleaned_csv_filename)

        # Map columns if needed
        column_mapping = {
            "Audio Filepath": "path",
            "Transcription": "sentence",
        }
        self.df.rename(columns=lambda x: column_mapping.get(x.strip(), x.strip()), inplace=True)

        # Verify required columns
        if 'path' not in self.df.columns or 'sentence' not in self.df.columns:
            raise ValueError("CSV file must contain 'path' and 'sentence' columns.")


    def __len__(self):
        """Returns the total number of samples in the dataset."""
        return len(self.df)

    def __getitem__(self, index):
        """Returns a single data item at a given index."""
        # Get the row from the DataFrame
        row = self.df.iloc[index]

        # Construct the audio file path
        audio_path = os.path.join(self.audio_base_path, row['path'])

        # Get the corresponding sentence
        sentence = row['sentence']

        # Return a dictionary containing the audio path and sentence
        return {'audio_path': audio_path, 'sentence': sentence}

    @staticmethod
    def clean_csv(input_path, output_path):
        """
        Cleans a CSV file by:
        - Replacing tab characters with commas.
        - Filtering rows with inconsistent numbers of fields.
        """
        with open(input_path, "r") as infile:
            lines = infile.readlines()

        # Replace tabs with commas
        clean_lines = [line.replace("\t", ",").replace("lacuna-audios-train/asanti-twi/audios/", "").replace("lacuna-audios-test/asanti-twi/audios/", "") for line in lines]

        # Filter rows with the correct number of fields
        expected_fields = clean_lines[0].count(",") + 1
        valid_lines = [line for line in clean_lines if line.count(",") + 1 == expected_fields]

        # Write cleaned content to a new file
        with open(output_path, "w") as outfile:
            outfile.writelines(valid_lines)


# Dataset URLs and paths
train_zip_url = "https://fisd-dataset.s3.amazonaws.com/fisd-asanti-twi-90p.zip"
train_csv_filename = "fisd-asanti-twi-90p/data.csv"
train_audio_base_path = "fisd-asanti-twi-90p/audios"

test_zip_url = "https://fisd-dataset.s3.amazonaws.com/fisd-asanti-twi-10p.zip"
test_csv_filename = "fisd-asanti-twi-10p/data.csv"
test_audio_base_path = "fisd-asanti-twi-10p/audios"

# Load datasets
test_dataset = AsantiTwiDataset(test_zip_url, test_csv_filename, test_audio_base_path)
train_dataset = AsantiTwiDataset(train_zip_url, train_csv_filename, train_audio_base_path)



In [ ]:
testinglabels = "/content/drive/MyDrive/ABENA_Trained/Testlabels.csv"
df = pd.read_csv(testinglabels, delimiter="\t", names=["Index", "path","sentence","translation"], header=0)
display(df)

# Functions to use BERT Model

In [ ]:
import transformers.pipelines.pt_utils
def correct_asr_output(text):

    masked_text = text + " [MASK]"  # Add the mask token at the end
    corrected_text = corrector_robako(masked_text)
    corrected_word = corrected_text[0]['token_str']
    corrected_sentence = masked_text.replace("[MASK]", corrected_word)

    corrected_sequence = corrected_sentence.replace("[CLS]", "").replace("[SEP]", "").strip()
    # Extract the predicted token (ignoring the original input and [MASK])
    return corrected_sequence[0][0]


def transcribe_audio(audio_path):
    # Pass options as keyword arguments within transcribe
    model = whisper.load_model("tiny")
    result = model.transcribe(audio_path, language="en", without_timestamps=True)
    return result['text']

In [ ]:
# Example usage for selecting a random statement
import random
import os


# Select a random index within the dataset
random_index = random.randint(0, len(test_dataset) - 1)

# Get the data item at the random index
random_item = test_dataset[random_index]

# Access the audio path and sentence from the random item
audio_path = random_item['audio_path']
sentence = random_item['sentence']

print("Random Audio Path:", audio_path)
print("Random Sentence:", sentence)

import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", message="Some weights of the model checkpoint") #Was flooding the output

# Transcribe and correct the random audio
asr_output = transcribe_audio(audio_path)
print("ASR Output:", asr_output)

# Load the BERT Abena model for correction
from transformers import AutoTokenizer, AutoModelForMaskedLM, pipeline

tokenizer_abena = AutoTokenizer.from_pretrained("Ghana-NLP/abena-base-asante-twi-uncased")
corrector_model_abena = AutoModelForMaskedLM.from_pretrained("Ghana-NLP/abena-base-asante-twi-uncased")
corrector_abena = pipeline("fill-mask", model=corrector_model_abena, tokenizer=tokenizer_abena, device=0)

# Correct ASR output using Abena
corrected_output_abena = corrector_abena(asr_output.replace(" ", " [MASK] "))
print("Corrected Output (Abena):", corrected_output_abena)
print("\n\n\n")

# Load the BERT Robako model for correction
tokenizer_robako = AutoTokenizer.from_pretrained("Ghana-NLP/robako-base-asante-twi-uncased")
corrector_model_robako = AutoModelForMaskedLM.from_pretrained("Ghana-NLP/robako-base-asante-twi-uncased")
corrector_robako = pipeline(task = "fill-mask", model=corrector_model_robako, tokenizer=tokenizer_robako, device=0)

# Correct ASR output using Robako
#loads of errors here while trying to extract just the sentence, so tried different ways to get just the output
#masked_texts = [correct_asr_output(text) for text in asr_output.split()]
#corrected_output_robako = [corrector_robako(text)[0]['token_str'] for text in asr_output.split()]
#corrected_output_robako = " ".join(corrected_output_robako)
#corrected_output_robako = [corrector_robako(text + " [MASK]")[0]['token_str'] for text in asr_output.split()]
corrected_output_robako = [corrector_robako(text + " <mask>")[0]['sequence'] for text in asr_output.split()]
corrected_output_robako = " ".join(corrected_output_robako)
print("Corrected Output (Robako):", corrected_output_robako)


#Using BERT to correct errors

## Method 1: Language Modelling
Trying to use predicting the next work to correct it (later figured out it was not a meaningful way to get things done)


In [ ]:
#for language modelling

import pandas as pd
from datasets import Dataset
from transformers import BertTokenizer, BertForMaskedLM, Trainer, TrainingArguments


dataset = Dataset.from_pandas(train_dataset.df)
#dataset = Dataset.from_pandas(test_dataset)

# Load the tokenizer
tokenizer = BertTokenizer.from_pretrained("Ghana-NLP/abena-base-asante-twi-uncased")

# Tokenize the dataset
'''def tokenize_function(examples):
    # Specifying max_length here ensures all sequences are the same length
    return tokenizer(examples["sentence"], padding="max_length", truncation=True, max_length=128) # Added max_length'''

def tokenize_function(examples):
    # Specifying max_length here ensures all sequences are the same length
    # Create labels by replacing masked tokens with -100 (ignore index)
    inputs = tokenizer(examples["sentence"], padding="max_length", truncation=True, max_length=128, return_tensors="pt")  # Added max_length, return_tensors="pt"
    inputs["labels"] = inputs.input_ids.detach().clone()
    # create random array of floats with equal dims to input_ids
    rand = torch.rand(inputs.input_ids.shape)
    # mask random 15% where token is not 0 [PAD], 1 [CLS], or 2 [SEP]
    mask_arr = (rand < 0.15) * (inputs.input_ids != 0) * (inputs.input_ids != 1) * (inputs.input_ids != 2)
    # loop through each row in input_ids tensor (cannot do in parallel)
    for i in range(inputs.input_ids.shape[0]):
        # get indices of mask positions from mask array
        selection = torch.flatten(mask_arr[i].nonzero()).tolist()
        # mask input_ids
        inputs.input_ids[i, selection] = tokenizer.mask_token_id
    # where input_ids is not masked, set labels to -100
    inputs["labels"][~mask_arr] = -100

    return inputs

tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.train_test_split(test_size=0.2)

# Load the model
model = BertForMaskedLM.from_pretrained("Ghana-NLP/abena-base-asante-twi-uncased")

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=6,
    weight_decay=0.01,
    report_to="none"
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
)

# Train the model
trainer.train()

trainer.save_model("/content/drive/MyDrive/ABENA_Trained2")
tokenizer.save_pretrained("/content/drive/MyDrive/ABENA_Trained2")

In [ ]:
trainer.save_model("/content/drive/MyDrive/ABENA_Trained")
tokenizer.save_pretrained("/content/drive/MyDrive/ABENA_Trained")

In [ ]:
from transformers import BertTokenizer, BertForMaskedLM, pipeline

# Load your fine-tuned model and tokenizer
model_path = "/content/drive/MyDrive/ABENA_Trained"  # Update with your save path

# Load the tokenizer from the configuration file
tokenizer = BertTokenizer.from_pretrained(model_path)

# Load the model, potentially specifying the safetensors file
model = BertForMaskedLM.from_pretrained(model_path, torch_dtype="auto")

# Create a fill-mask pipeline
fill_mask = pipeline("fill-mask", model=model, tokenizer=tokenizer)

# Test with a sentence
test_sentence = "Mɛtumi ayɛ [MASK] million cedis."
predictions = fill_mask(test_sentence)

# Print the predictions
for prediction in predictions:
    print(f"Predicted word: {prediction['token_str']}, Score: {prediction['score']:.4f}")


# Print only the predicted word
predicted_word = predictions[0]['token_str']
print(f"Predicted sentence: {test_sentence.replace('[MASK]', predicted_word)}")

In [ ]:
import pandas as pd
import random

def mask_random_word(sentence):
    """Masks a single random word in a sentence.

    Args:
        sentence (str): The input sentence.

    Returns:
        str: The sentence with one word randomly masked.
    """
    words = sentence.split()
    if words:  # Check if the sentence is not empty
        random_index = random.randint(0, len(words) - 1)  # Get a random index
        words[random_index] = "[MASK]"  # Replace the word at that index
    return " ".join(words)  # Join the words back into a sentence

In [ ]:
import pandas as pd # If using a CSV file
from transformers import BertTokenizer, BertForMaskedLM, pipeline

# Load fine-tuned model and tokenizer
model_path = "/content/drive/MyDrive/ABENA_Trained"
tokenizer = BertTokenizer.from_pretrained(model_path)
model = BertForMaskedLM.from_pretrained(model_path)
fill_mask = pipeline("fill-mask", model=model, tokenizer=tokenizer)


testinglabels = "/content/drive/MyDrive/ABENA_Trained2/Testlabels.csv"
df = pd.read_csv(testinglabels, delimiter="\t", names=["Index", "path","sentence","translation"], header=0)
df["masked_sentence"] = df["sentence"].apply(mask_random_word)


# Prediction loop
#for sentence in sentences:
for index, row in df.iterrows():
    # Replace a word with [MASK] for prediction
    masked_sentence = row["masked_sentence"]
    predictions = fill_mask(masked_sentence)
    #masked_sentence = sentence.replace("target_word", "[MASK]")
    #predictions = fill_mask(masked_sentence)
    predicted_word = predictions[0]['token_str']  # Get the top prediction

    # Compare predicted word with actual word
    print(f"Original Sentence: {row['sentence']}")
    print(f"Masked Sentence: {masked_sentence}")
    print(f"Predicted Word: {predicted_word}")
    print(f"Predicted Sentence: {masked_sentence.replace('[MASK]', predicted_word)}")
    print("-" * 20)  # Separator

##Method 2: Text Correction
This is the actual way to do correct the Whisper output, but had insufficient data for it. Was using the error data rom the finetuning of the whisper model on the financial inclusion dataset.

In [ ]:
import pandas as pd
from datasets import Dataset
from transformers import BertTokenizer, EncoderDecoderModel, Trainer, TrainingArguments


from transformers import BertTokenizer, BertForMaskedLM, pipeline

# Load your fine-tuned model and tokenizer
model_path = "/content/drive/MyDrive/ABENA_Trained"
tokenizer = BertTokenizer.from_pretrained(model_path)
model = BertForMaskedLM.from_pretrained(model_path)
fill_mask = pipeline("fill-mask", model=model, tokenizer=tokenizer)

In [ ]:
testinglabels = "/content/drive/MyDrive/ABENA_Trained/hypotheses_references.csv"
dataset = pd.read_csv(testinglabels, names=["Index", "path","sentence","translation"], header=0)

In [ ]:
!pip install datasets transformers


# Load a pre-trained BERT model for Asante Twi
model_name = "Ghana-NLP/abena-base-asante-twi-uncased"  # Or "Ghana-NLP/robako-base-asante-twi-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForMaskedLM.from_pretrained(model_name)

fill_mask = pipeline("fill-mask", model=model, tokenizer=tokenizer)

# Example test sentence
test_sentence = "Mɛtumi ayɛ [MASK] million cedis."
predictions = fill_mask(test_sentence)
print(f"Predicted sentence: {test_sentence.replace('[MASK]', predictions[0]['token_str'])}")

In [ ]:
# Load dataset
data_path = "/content/drive/MyDrive/ABENA_Trained/hypotheses_references.csv"
df = pd.read_csv(data_path, names=["incorrect", "correct", "incorrect cleaned", "correct cleaned"], header=0)
dataset = Dataset.from_pandas(df)

# Tokenization function
def preprocess_function(examples):
    # Use the 'incorrect cleaned' and 'correct cleaned' columns
    inputs = tokenizer(examples["incorrect cleaned"], padding="max_length", truncation=True, max_length=128)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["correct cleaned"], padding="max_length", truncation=True, max_length=128)
    inputs["labels"] = labels["input_ids"]
    return inputs

tokenized_datasets = dataset.map(preprocess_function, batched=True)

# Split into train, validation, and test sets
from datasets import DatasetDict # Import DatasetDict here
train_testvalid = tokenized_datasets.train_test_split(test_size=0.2)
test_valid = train_testvalid['test'].train_test_split(test_size=0.5)
tokenized_datasets = DatasetDict({
    'train': train_testvalid['train'],
    'test': test_valid['test'],
    'validation': test_valid['train']})


# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    save_total_limit = 2, # Only last 2 models are saved. Older ones are deleted.
    load_best_model_at_end=True,
    save_strategy="epoch",
    report_to="none",
    logging_steps = 10
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"]
)

# Train the model
trainer.train()

# Save the fine-tuned model
trainer.save_model("/content/drive/MyDrive/ABENA_FineTuned")
tokenizer.save_pretrained("/content/drive/MyDrive/ABENA_FineTuned")

In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM, pipeline

# Load the fine-tuned model
fine_tuned_tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/ABENA_FineTuned")
fine_tuned_model = AutoModelForMaskedLM.from_pretrained("/content/drive/MyDrive/ABENA_FineTuned")

corrector = pipeline("fill-mask", model=fine_tuned_model, tokenizer=fine_tuned_tokenizer)

def correct_text(text):
    """Corrects the input text using the fine-tuned model."""
    # Tokenize the input text
    inputs = fine_tuned_tokenizer(text, return_tensors="pt")
    input_ids = inputs["input_ids"][0]  # Access the first element of the tensor

    # Create a copy of the input_ids to avoid modifying the original
    masked_input_ids = input_ids.clone()

    # Randomly mask 15% of the tokens (excluding special tokens)
    rand = torch.rand(input_ids.shape)
    mask_arr = (rand < 0.15) * (input_ids != fine_tuned_tokenizer.cls_token_id) * (input_ids != fine_tuned_tokenizer.sep_token_id)

    # Apply the mask to the selected tokens
    masked_input_ids[mask_arr] = fine_tuned_tokenizer.mask_token_id

    # Run the model to predict the masked tokens
    outputs = fine_tuned_model(masked_input_ids.unsqueeze(0))  # Add batch dimension
    predictions = torch.argmax(outputs.logits, dim=-1)

    # Decode the predicted tokens to get the corrected text
    corrected_text = fine_tuned_tokenizer.decode(predictions[0], skip_special_tokens=True)

    return corrected_text


In [ ]:
# Example usage:
text_to_correct = "Me den de Kwoku"
corrected_text = correct_text(text_to_correct)
print(f"Original Text: {text_to_correct}")
print(f"Corrected Text: {corrected_text}")